In [1]:
import os
import numpy as np # to process the data
import random
import math
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
class FlowerDataset:
    def __init__(self, directory):
        self.directory = directory
        self.images = []
        self.labels = []

    def load_data(self):
        for i, img in enumerate(os.listdir(self.directory)):
            if i >= 10:  # only load the first 10 images
                break
            try:
                img_path = os.path.join(self.directory, img)
                with Image.open(img_path) as img_array:  # read the image
                    gray_array = img_array.convert('L')  # ensure image is grayscale
                    resized_array = gray_array.resize((150, 150))  # resize the image
                    self.images.append([list(resized_array.getdata())[i:i+150] for i in range(0, 22500, 150)])  # keep image in 2D
                    label = img.split('_')[0]  # extract label from filename
                    self.labels.append(label)
            except Exception as e:
                print(f"Error loading image {img}: {e}")

    def preprocess_data(self):
        # Normalize pixel values
        self.images = [[[pixel / 255.0 for pixel in row] for row in image] for image in self.images]
        print(f"Loaded {len(self.images)} images")

# Load and preprocess the data
dataset = FlowerDataset('flowers')
dataset.load_data()
dataset.preprocess_data()

# Print the shape of the preprocessed images
print(f"Shape of the preprocessed images: {len(dataset.images)} images, each with {len(dataset.images[0])}x{len(dataset.images[0][0])} pixels")

# Dimensions of the first preprocessed image
print(f"Dimensions of the first preprocessed image: {len(dataset.images[0])}x{len(dataset.images[0][0])} pixels")

# Split the data into training, validation, and test sets
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# Custom function to split the dataset
def custom_train_test_split(data, labels, train_size):
    train_count = int(len(data) * train_size)
    return data[:train_count], data[train_count:], labels[:train_count], labels[train_count:]

# train is now 75% of the entire data set
x_train, x_temp, y_train, y_temp = custom_train_test_split(dataset.images, dataset.labels, train_ratio)

# test is now 10% of the initial data set, validation is now 15%
val_count = int(len(x_temp) * (validation_ratio / (test_ratio + validation_ratio)))
x_val, x_test = x_temp[:val_count], x_temp[val_count:]
y_val, y_test = y_temp[:val_count], y_temp[val_count:]

# Print the sizes of the datasets
print(f"Train set size: {len(x_train)}, Validation set size: {len(x_val)}, Test set size: {len(x_test)}")

# Print the unique classes in the training set
unique_classes = set(y_train)
print(f"Unique classes in the training set: {unique_classes}")

Loaded 10 images
Shape of the preprocessed images: 10 images, each with 150x150 pixels
Dimensions of the first preprocessed image: 150x150 pixels
Train set size: 7, Validation set size: 1, Test set size: 2
Unique classes in the training set: {'orchids', 'hibiscus', 'daisies', 'hydrangeas', 'lilies', 'peonies', 'tulip'}


In [17]:
def relu(x):
    if isinstance(x, list):
        return [max(0, xi) for xi in x]
    else:
        return max(0, x)

def relu_derivative(x):
    if isinstance(x, list):
        return [1 if xi > 0 else 0 for xi in x]
    else:
        return 1 if x > 0 else 0

def mean_squared_error(y_true, y_pred):
    return sum((yt - yp) ** 2 for yt, yp in zip(y_true, y_pred)) / len(y_true)

def gradient_mse(y_true, y_pred):
    return [2 * (yp - yt) for yt, yp in zip(y_true, y_pred)]

def softmax(x):
    exps = [math.exp(i) for i in x]
    sum_of_exps = sum(exps)
    return [j / sum_of_exps for j in exps]

def cross_entropy(predictions, targets):
    # Assuming predictions is a list of probabilities and targets is a list of one-hot encoded classes
    N = len(predictions)
    ce = 0
    for p, t in zip(predictions, targets):
        ce -= t * math.log(p) if p > 0 else 0  # Adding check to prevent math domain error
    ce /= N
    return ce

def reshape_input(flat_input, image_height, image_width):
    return [flat_input[i * image_width:(i + 1) * image_width] for i in range(image_height)]

# Layer classes
class Layer:
    def forward(self, input):
        raise NotImplementedError

    def backward(self, input, gradient):
        raise NotImplementedError

class Conv2D(Layer):
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = [[random.random() for _ in range(filter_size * filter_size)] for _ in range(num_filters)]

    def forward(self, input):
        # Assume input is a 2D list representing the image
        height, width = len(input), len(input[0])
        output_height = height - self.filter_size + 1
        output_width = width - self.filter_size + 1
        # Initialize output with zeros
        output = [[0 for _ in range(output_width)] for _ in range(output_height)]

        for i in range(output_height):
            for j in range(output_width):
                for f in range(self.num_filters):
                    filter_output = 0
                    for m in range(self.filter_size):
                        for n in range(self.filter_size):
                            filter_output += input[i + m][j + n] * self.filters[f][m * self.filter_size + n]
                    output[i][j] = filter_output
        return output

    def backward(self, input, gradient):
        d_filters = [[[0 for _ in range(self.filter_size * self.filter_size)] for _ in range(self.num_filters)]]
        for i in range(len(input) - self.filter_size + 1):
            for j in range(len(input[0]) - self.filter_size + 1):
                for f in range(self.num_filters):
                    for m in range(self.filter_size):
                        for n in range(self.filter_size):
                            d_filters[f][m * self.filter_size + n] += input[i + m][j + n] * gradient[i][j]
        return d_filters

class MaxPooling2D(Layer):
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, input):
        # Assume input is a 2D list representing the image
        height, width = len(input), len(input[0])
        new_height = height // self.pool_size
        new_width = width // self.pool_size
        # Initialize output with zeros
        output = [[0 for _ in range(new_width)] for _ in range(new_height)]

        for i in range(new_height):
            for j in range(new_width):
                pool = [input[i * self.pool_size + m][j * self.pool_size + n]
                        for m in range(self.pool_size)
                        for n in range(self.pool_size)]
                output[i][j] = max(pool)
        return output

    def backward(self, input, gradient):
        d_input = [[0 for _ in range(len(input[0]))] for _ in range(len(input))]
        for i in range(0, len(input), self.pool_size):
            for j in range(0, len(input[0]), self.pool_size):
                window = [input[i + m][j + n] for m in range(self.pool_size) for n in range(self.pool_size)]
                max_value = max(window)
                for m in range(self.pool_size):
                    for n in range(self.pool_size):
                        if input[i + m][j + n] == max_value:
                            d_input[i + m][j + n] = gradient[i // self.pool_size][j // self.pool_size]
        return d_input

class Flatten(Layer):
    def forward(self, input):
        self.input_shape = (len(input), len(input[0]))  # save this for backward pass
        return [item for sublist in input for item in sublist]  # flatten the list

    def backward(self, input, gradient):
        return [gradient[i:i+self.input_shape[1]] for i in range(0, len(gradient), self.input_shape[1])]

class Dense(Layer):
    def __init__(self, num_inputs, num_outputs):
        # Initialize weights and biases
        self.weights = [[random.random() for _ in range(num_inputs)] for _ in range(num_outputs)]
        self.biases = [random.random() for _ in range(num_outputs)]

    def forward(self, input):
        # Compute the weighted sum of inputs plus biases
        return [sum(i * w + b for i, w in zip(input, weights_row)) for weights_row, b in zip(self.weights, self.biases)]

    def backward(self, input, gradient):
        # Compute gradients with respect to weights and biases
        d_weights = [[i * g for i in input] for g in gradient]
        d_biases = gradient
        d_input = [0 for _ in range(len(input))]
        for i, g in enumerate(gradient):
            for j, w in enumerate(self.weights[i]):
                d_input[j] += g * w
        return d_input, d_weights, d_biases

class Dropout(Layer):
    def __init__(self, rate):
        self.rate = rate

    def forward(self, input):
        self.input_shape = (len(input), len(input[0]))  # save this for backward pass
        self.mask = [[random.random() > self.rate for _ in range(self.input_shape[1])] for _ in range(self.input_shape[0])]
        return [[input[i][j] * self.mask[i][j] for j in range(self.input_shape[1])] for i in range(self.input_shape[0])]

    def backward(self, input, gradient):
        return [[gradient[i][j] * self.mask[i][j] for j in range(self.input_shape[1])] for i in range(self.input_shape[0])]

# Activation functions as layers
class ReLU(Layer):
    def forward(self, input):
        output = [relu(x) for x in input]
        # print(f"ReLU output shape: {len(output)}x{len(output[0])}, type: {type(output)}")  # Add this line
        return output

    def backward(self, input, gradient):
        # Apply the derivative of ReLU to the gradient
        return [g * relu_derivative(x) for x, g in zip(input, gradient)]

# Loss function as a class
class SoftmaxCrossEntropyLoss:
    def forward(self, logits, labels):
        self.predictions = softmax(logits)
        return cross_entropy(self.predictions, labels)

    def backward(self, logits, labels):
        return [p - l for p, l in zip(self.predictions, labels)]

In [18]:
class CustomCNNModel:
    def __init__(self):
        self.layers = [
            
        ]
        self.loss = SoftmaxCrossEntropyLoss()

    def forward(self, input, labels):
        for layer in self.layers:
            input = layer.forward(input)
        # Apply softmax to the logits to get probabilities
        probabilities = softmax(input)
        return self.loss.forward(probabilities, labels)

    def backward(self, logits, labels):
        gradient = self.loss.backward(logits, labels)
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)

    def train(self, x_train, y_train, x_val, y_val, epochs, learning_rate, threshold, early_stopping):
        history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
        best_val_loss = float('inf')
        no_improvement = 0
    
        for epoch in range(epochs):
            # Training phase
            train_loss = 0
            train_correct = 0
            for x, y in zip(x_train, y_train):
                logits = self.forward(x, y)
                # Apply softmax to the logits to get probabilities
                probabilities = softmax(logits)
                train_loss += self.loss.forward(probabilities, y)
                train_correct += self.argmax(logits) == self.argmax(y)
                gradient = self.loss.backward(logits, y)
                for layer in reversed(self.layers):
                    if isinstance(layer, Dense):
                        gradient, d_weights, d_biases = layer.backward(x, gradient)
                        # Update weights and biases
                        layer.weights = [[w - learning_rate * dw for w, dw in zip(weights_row, d_weights_row)] for weights_row, d_weights_row in zip(layer.weights, d_weights)]
                        layer.biases = [b - learning_rate * db for b, db in zip(layer.biases, d_biases)]
                    else:
                        gradient = layer.backward(x, gradient)
                    
            train_loss /= len(x_train)
            train_acc = train_correct / len(x_train)
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)

            # Validation phase
            val_loss = 0
            val_correct = 0
            for x, y in zip(x_val, y_val):
                logits = self.forward(x, y)
                val_loss += self.loss.forward(logits, y)
                val_correct += self.argmax(logits) == self.argmax(y)

            val_loss /= len(x_val)
            val_acc = val_correct / len(x_val)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)

            # Print training and validation results
            # Print loss and accuracy for this epoch
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}")
            print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}")

            # Early stopping
            if val_loss < best_val_loss - threshold:
                best_val_loss = val_loss
                no_improvement = 0
            else:
                no_improvement += 1
            if no_improvement >= early_stopping:
                print(f"Stopping training after {epoch+1} epochs due to no improvement in validation loss.")
                break

        return history

    def argmax(self, logits):
        # Implement argmax without NumPy
        return max(range(len(logits)), key=lambda i: logits[i])



# Example usage of the model
flat_input = [0.0] * 22500  # Example flat input list
image_height = 150  # The height of the image
image_width = 150  # The width of the image
input_2d = reshape_input(flat_input, image_height, image_width)

# Define your training parameters
epochs = 10  # Number of epochs to train for
learning_rate = 1  # Learning rate
threshold = 0.01  # Threshold for early stopping
early_stopping = 10  # Number of rounds without improvement before early stopping

# Create an instance of the model
model = CustomCNNModel()

# Train the model
history = model.train(x_train, y_train, x_val, y_val, epochs, learning_rate, threshold, early_stopping)

TypeError: must be real number, not list